In [ ]:
#Python standard library imports
import datetime as dt
import pathlib
import re
#Scientific Python ecosystem imports
import pandas as pd
import numpy as np
#Text mining packages
import nltk as nltk
from whoosh.lang.porter import stem
#MM Import
import datetime as dt
import pathlib
import logging
import copy
import unicodedata

from docx2python import docx2python
from bs4 import BeautifulSoup
import html2text
import urllib.request

#from polmap.polmap import preprocess_text, doc2text # replaced the keyword processing block

In [ ]:
def prepare_keywords(a_string, stop_words, exception_dict=None):
    """
    Prepare text for mapping.
    """
    text_string = a_string

    if text_string==None: #this should be moved to the prepare keywords wrapper function
        return None
    # if text_string is not str:
    #     raise TypeError('text_string is not a string') 
    #     #How to return the name of the variable passed by user with format?
    # Get error when using it with apply and lambda in pandas
    
    if exception_dict==None:
        exception_dict = {"aids": "ai&ds&",
                          "productivity": "pro&ductivity&",
                          "remittances" : "remit&tance&"                 
                          }
    elif exception_dict is not dict:
        raise TypeError('exception_dict is not a dict')
    
    reverse_exception_dict = {value : key for key, value in exception_dict.items()}

    #stem_fun = lambda rgx_str: stem(rgx_str) if rgx_str not in rgx_str else rgx_str
        
    #remove all from stop_words to keep in keywords.
    # Review scoping rules in python, this fails with:
    # NameError: name 'stop_words' is not defined when called in lambda function
    # I would expect the variable to always exist whenever calling the function, but it does not.
    #if stop_words==None:
    #    stop_words = set(nltk.corpus.stopwords.words("english"))
    #    stop_words.remove("all")

       
    #text_string = text_string.replace('\xa0',' ') #Remove some weird \xa0 characters

    text_string = text_string.lower().strip()
    
    text_string = re.sub(r'[^a-zA-Z- ]+', '', text_string)
    
    text_string = re.sub(r'([\w-]+)', r' \1 ', text_string) #Equivalent to center, adds leading and trailing space to the captured group
    
    text_string = text_string.replace(' rd ', ' R&D ')
    
    text_string = re.sub(r'([\w-]{3,}|ph)', r'\1', text_string)
    
    # not sure this is working the way intended, 
    # if the plan was to drop two characters words,
    # it is not  as we are however counting also spaces.
    # an easy fix would be to move it before the centering of the terms
    
    for key, value in exception_dict.items(): #Protect exceptions from stemming
        text_string = text_string.replace(key, value)
    
    for word in stop_words: #Remove stopwords
        text_string = text_string.replace(' '+word+' ', '') 
    
    text_string = re.sub(r'[a-zA-Z-&]+', 
    lambda rgx_word: ' '+stem(rgx_word.group())+' ',# if not rgx_word in stop_words else rgx_word, 
    text_string) #if not rgx_word in stop_words if rgx_word != "aids",
        
    for key, value in exception_dict.items(): #Restore words from exception protection
        text_string = text_string.replace(value, key)
    
    text_string = ' '+text_string+' '
    
    text_string = re.sub(r' {2,}', r' ', text_string) #Remove multiple spaces
        
    return text_string

In [ ]:
def preprocess_text(text_string, stop_words, exception_dict=None):
    """
    Prepare text for mapping.
    """
    
    # if text_string is not str:
    #     raise TypeError('text_string is not a string') 
    #     #How to return the name of the variable passed by user with format?
    # Get error when using it with apply and lambda in pandas
    
    if exception_dict==None:
        exception_dict = {"aids": "ai&ds&",
                          "productivity": "pro&ductivity&",
                          "remittances" : "remit&tance&"                 
                          }
    elif exception_dict is not dict:
        raise TypeError('exception_dict is not a dict')
    
    reverse_exception_dict={value : key for key, value in exception_dict.items()}
    
    #remove all from stop_words to keep in keywords.
    # Review scoping rules in python, this fails with:
    # NameError: name 'stop_words' is not defined when called in lambda function
    # I would expect the variable to always exist whenever calling the function, but it does not.
    #if stop_words==None:
    #    stop_words = set(nltk.corpus.stopwords.words("english"))
    #    stop_words.remove("all")
 

    text_string = text_string.split(";")
    
    text_list = map(lambda term: term.split(),  text_string)    
    
    text_list = [ [term.lower().strip() for term in terms]
                      for terms in text_list ]    

    text_list = [ [re.sub(r"[^a-zA-Z- ]+", '', term) for term in terms]
                      for terms in text_list ]    

    text_list = [ [term.center(len(term)+2) for term in terms]
                      for terms in text_list ]    

    text_list = [ [term.replace(" rd ", "R&D") for term in terms]
                      for terms in text_list ]    

    text_list = [ [term for term in terms if len(term) > 2 or term == "ph" ]
                      for terms in text_list ]
    
    # not sure this is working the way intended, 
    # if the plan was to drop two characters words,
    # it is not  as we are however counting also spaces.
    # an easy fix would be to move it before the centering of the terms
    text_list = [ [term.strip(' ') for term in terms]
                      for terms in text_list ]    
    
    text_list = [ [exception_dict[term] if term in exception_dict.keys() 
                      else term
                      for term in terms]
                      for terms in text_list ]    
    
    text_list = [ [ stem(term) for term in terms 
                      if not term in stop_words if term != "aids" ]
                      for terms in text_list ]    
    
    text_list = [ [reverse_exception_dict[term] if term in reverse_exception_dict.keys() 
                      else term
                      for term in terms]
                      for terms in text_list ]    
        
    text_list = [ ' '.join(terms) for terms in text_list ]    

    text_list = [ ' '+terms+' ' for terms in text_list ]    
    
    text_list = [terms for terms in text_list if terms!='  ']
    
    return text_list

In [ ]:
######################################
########### 2) MM Read the list of keywords and apply the prepare_keyords text processing function from polmap

keys = pd.read_excel('keys_update_27012020.xlsx', sheet_name = 'Target_keys' ) #MM 'keys_from_RAKE-GBV_DB_SB_v3.xlsx', sheet_name= 'Sheet1'

stop_words = set(nltk.corpus.stopwords.words("english"))
stop_words.remove("all")

raw_keywords_df=keys['Keys'].str.split(';', expand=True)

keys['Keys']=keys['Keys'].apply(lambda x: preprocess_text(x, stop_words))




In [ ]:
print(stop_words)

In [ ]:
keywords = keys['Keys'].apply(pd.Series)

keywords.replace(to_replace={'':None, np.NaN:None}, inplace=True)


In [ ]:
raw_keywords_df.replace(to_replace={'':None, np.NaN:None}, inplace=True)
raw_keywords_df.drop(labels=57, axis=1, inplace=True)


In [ ]:

processed_keywords_df=raw_keywords_df.applymap(lambda x: prepare_keywords(x, stop_words))
processed_keywords_df.replace(to_replace={'':None, np.NaN:None}, inplace=True)


In [ ]:
new_vs_standard=processed_keywords_df.compare(keywords, keep_shape=True)
raw_vs_new = raw_keywords_df.compare(processed_keywords_df, keep_shape=True)
raw_vs_standard = raw_keywords_df.compare(keywords, keep_shape=True)
#comparison[1][1]='=IF(AND(ISBLANK(New_preprocessing!B2),ISBLANK(Standard_processing!B2)),"",IF(New_preprocessing!C2=Standard_preprocessing!C2,TRUE,FALSE))'


In [ ]:
keywords.head()

In [ ]:
processed_keywords_df.head()

In [ ]:
raw_keywords_df.head()

In [ ]:
with pd.ExcelWriter('updated_processed_keywords_test.xlsx', engine='xlsxwriter') as writer:
    raw_keywords_df.to_excel(writer, sheet_name='Raw_keywords')
    processed_keywords_df.to_excel(writer, sheet_name='New_preprocessing')
    keywords.to_excel(writer, sheet_name='Standard_preprocessing')
    new_vs_standard.to_excel(writer, sheet_name='New_vs_Standard')
    raw_vs_new.to_excel(writer, sheet_name='Raw_vs_New')
    raw_vs_standard.to_excel(writer, sheet_name='Raw_vs_Standard')
